In [1]:
import json
from datetime import datetime, date
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
def get_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ").date()

def convert_time_to_seconds(time_str):
    time = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ").time()
    return time.hour * 3600 + time.minute * 60 + time.second

In [3]:
def get_game_info(game,tournament,region,stage):
    game_info = game[0]
    game_end = game[-1]
    
    blue = pd.DataFrame({"side":["blue"]})
    red = pd.DataFrame({"side":["red"]})
    
    start_time = convert_time_to_seconds(game_info["eventTime"])
    end_time = convert_time_to_seconds(game_end["eventTime"])
    date = get_date(game_info["eventTime"])

    blue["date"] = date
    blue["start_time"] = start_time
    red["date"] = date
    red["start_time"] = start_time

    blue["end_time"] = end_time
    red["end_time"] = end_time

    blue["duration"] = blue["end_time"] - blue["start_time"]
    red["duration"] = blue["duration"]
    
    patch = game_info["gameVersion"]
    blue["patch"] =patch
    red["patch"] =patch
    
    participants =game_info["participants"]
    roles = ["top","jng","mid","adc","sup","top","jng","mid","adc","sup"]
    
    for player,role in zip(participants,roles):
        summonerName =player["summonerName"].split(" ")
        team_tag = summonerName.pop(0)
        player_name = " ".join(summonerName)
        if player["teamID"]==100:
            side = blue
        else:
            side = red
        side["team"]= team_tag
        side[role]= player_name
        if not team_tag in regions or regions[team_tag]==None:
            regions[team_tag]=region
    blue["opponent"]= red["team"]
    red["opponent"]= blue["team"]
    for role in ["top","jng","mid","adc","sup"]:
        blue[f"enemy_{role}"]=red[role]
        red[f"enemy_{role}"]=blue[role]
    blue_tag = blue["team"][0]
    red_tag = red["team"][0]
    blue["region"]=regions.get(blue_tag)
    red["region"]=regions.get(red_tag)
    blue["opponent_region"]=regions.get(red_tag)
    red["opponent_region"]=regions.get(blue_tag)
    blue["stage"]=stage
    red["stage"]=stage
    

    if game_end["winningTeam"]== 100:
        blue["result"]=1
        red["result"]=0
    else:
        blue["result"]=0
        red["result"]=1
    return(blue,red)

In [4]:
def get_status_updates(game,blue,red):
    start_time = blue["start_time"]
    roles = ["top","jng","mid","adc","sup","top","jng","mid","adc","sup"]
    end_time = int(convert_time_to_seconds(game[-2]["eventTime"]) - start_time)
    for event in game:
        event_time = int(convert_time_to_seconds(event["eventTime"]) - start_time)
        for time in [end_time]: #can add more times, 300 sec for 5 mins game time
            if (event["eventType"] == "stats_update") and (event_time == int(time)):
                if time == end_time:
                    time = "end"
                participants = event["participants"]
                variables =["totalGold","level", "shutdownValue"]
                stats= ["MINIONS_KILLED","NEUTRAL_MINIONS_KILLED","NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE","NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE","CHAMPIONS_KILLED","NUM_DEATHS", "ASSISTS", "VISION_SCORE", "WARD_KILLED", "WARD_PLACED", "TOTAL_DAMAGE_DEALT", "TOTAL_DAMAGE_DEALT_TO_CHAMPIONS", "TOTAL_DAMAGE_TAKEN", "BASIC_PINGS", "COMMAND_PINGS","DANGER_PINGS","GET_BACK_PINGS","RETREAT_PINGS", "ON_MY_WAY_PINGS", "ASSIST_ME_PINGS", "ENEMY_MISSING_PINGS", "PUSH_PINGS", "ALL_IN_PINGS", "HOLD_PINGS", "BAIT_PINGS", "VISION_CLEARED_PINGS", "ENEMY_VISION_PINGS", "NEED_VISION_PINGS"]
                team_stats = ["inhibKills","towerKills","baronKills","dragonKills"]
                for player,role in zip(participants,roles):
                    if player["teamID"]==100:
                        side = blue
                    else:
                        side = red
                    for var in variables:
                        side[f"{role}_{var}_{time}"] = player[var]
            
                    stat_list = player["stats"]
                    for stat in stat_list:
                        if stat["name"] in stats:
                            name = stat["name"]
                            side[f"{role}_{name}_{time}"] = stat["value"]
                for stat in team_stats:
                    red[f"{stat}_{time}"] =event["teams"][1][stat]
                    blue[f"{stat}_{time}"] =event["teams"][0][stat]
    return(blue,red)

In [5]:
#dictionary for team regions
regions = {}
df = pd.DataFrame()
directory = "C:/GLOBAL POWER RANKINGS/games/"
for folder in os.listdir(directory):
    tournament = folder.split("_")[0]
    if tournament != "msi" or tournament !="worlds" or tournament !="midseason":
        #setting the region for teams who play in the tournaments (excluding international events)
        region = tournament
    else:
        region = None
    for stage in os.listdir(f"{directory}{folder}"):
        print(f" processing {directory}{folder}/{stage}")
        for games in os.listdir(f"{directory}{folder}/{stage}"):
            with open(f"{directory}{folder}/{stage}/{games}", "r") as json_file:
                game = json.load(json_file)
                blue,red = get_game_info(game,tournament,region,stage)
                blue,red = get_status_updates(game,blue,red)
                df =pd.concat([df,blue,red])
                
df.head()

 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Groups
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Playoffs
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Regular Season


,side,date,start_time,end_time,duration,patch,team,top,jng,mid,...,sup_ALL_IN_PINGS_end,sup_HOLD_PINGS_end,sup_BAIT_PINGS_end,sup_VISION_CLEARED_PINGS_end,sup_ENEMY_VISION_PINGS_end,sup_NEED_VISION_PINGS_end,inhibKills_end,towerKills_end,baronKills_end,dragonKills_end
0,blue,2023-04-08,61881,63897,2016,13.6.500.291,MAD,Chasy,Elyoya,Nisqy,...,1,0,0,0,1,0,0,3,0,2
0,red,2023-04-08,61881,63897,2016,13.6.500.291,VIT,Photon,Bo,Perkz,...,0,0,0,0,14,0,1,7,1,2
0,blue,2023-04-08,68967,70780,1813,13.6.500.291,AST,Finn,113,LIDER,...,1,1,1,0,24,0,0,3,0,0
0,red,2023-04-08,68967,70780,1813,13.6.500.291,FNC,Oscarinin,Razork,Humanoid,...,0,1,0,0,0,0,2,10,2,4
0,blue,2023-04-09,61148,62895,1747,13.6.500.291,SK,Irrelevant,Markoon,Sertuss,...,0,1,0,0,0,0,0,5,0,0


In [6]:
pd.pivot_table(df, index = 'result', columns = 'team', values = 'region' ,aggfunc ='count')

team,AST,BDS,FNC,G2,KOI,MAD,SK,TH,VIT,XL
result,,,,,,,,,,
0,8,5,9,9,10,14,9,7,9,7
1,8,16,6,13,7,17,4,3,11,2


In [8]:
ping_cols = [col for col in df.columns if "PINGS_end" in col]
print(ping_cols)
df["total_pings"]= df[ping_cols].sum(axis =1)
df.drop(columns =ping_cols)
df.head()

['top_BASIC_PINGS_end', 'top_COMMAND_PINGS_end', 'top_DANGER_PINGS_end', 'top_GET_BACK_PINGS_end', 'top_RETREAT_PINGS_end', 'top_ON_MY_WAY_PINGS_end', 'top_ASSIST_ME_PINGS_end', 'top_ENEMY_MISSING_PINGS_end', 'top_PUSH_PINGS_end', 'top_ALL_IN_PINGS_end', 'top_HOLD_PINGS_end', 'top_BAIT_PINGS_end', 'top_VISION_CLEARED_PINGS_end', 'top_ENEMY_VISION_PINGS_end', 'top_NEED_VISION_PINGS_end', 'jng_BASIC_PINGS_end', 'jng_COMMAND_PINGS_end', 'jng_DANGER_PINGS_end', 'jng_GET_BACK_PINGS_end', 'jng_RETREAT_PINGS_end', 'jng_ON_MY_WAY_PINGS_end', 'jng_ASSIST_ME_PINGS_end', 'jng_ENEMY_MISSING_PINGS_end', 'jng_PUSH_PINGS_end', 'jng_ALL_IN_PINGS_end', 'jng_HOLD_PINGS_end', 'jng_BAIT_PINGS_end', 'jng_VISION_CLEARED_PINGS_end', 'jng_ENEMY_VISION_PINGS_end', 'jng_NEED_VISION_PINGS_end', 'mid_BASIC_PINGS_end', 'mid_COMMAND_PINGS_end', 'mid_DANGER_PINGS_end', 'mid_GET_BACK_PINGS_end', 'mid_RETREAT_PINGS_end', 'mid_ON_MY_WAY_PINGS_end', 'mid_ASSIST_ME_PINGS_end', 'mid_ENEMY_MISSING_PINGS_end', 'mid_PUSH_PIN

,side,date,start_time,end_time,duration,patch,team,top,jng,mid,...,sup_HOLD_PINGS_end,sup_BAIT_PINGS_end,sup_VISION_CLEARED_PINGS_end,sup_ENEMY_VISION_PINGS_end,sup_NEED_VISION_PINGS_end,inhibKills_end,towerKills_end,baronKills_end,dragonKills_end,total_pings
0,blue,2023-04-08,61881,63897,2016,13.6.500.291,MAD,Chasy,Elyoya,Nisqy,...,0,0,0,1,0,0,3,0,2,603
0,red,2023-04-08,61881,63897,2016,13.6.500.291,VIT,Photon,Bo,Perkz,...,0,0,0,14,0,1,7,1,2,780
0,blue,2023-04-08,68967,70780,1813,13.6.500.291,AST,Finn,113,LIDER,...,1,1,0,24,0,0,3,0,0,517
0,red,2023-04-08,68967,70780,1813,13.6.500.291,FNC,Oscarinin,Razork,Humanoid,...,1,0,0,0,0,2,10,2,4,343
0,blue,2023-04-09,61148,62895,1747,13.6.500.291,SK,Irrelevant,Markoon,Sertuss,...,1,0,0,0,0,0,5,0,0,484


In [10]:
#df.to_csv("lec_spring_2023.csv",index=False)
df = df.drop(columns =ping_cols)
df.head()

,side,date,start_time,end_time,duration,patch,team,top,jng,mid,...,sup_WARD_KILLED_end,sup_VISION_SCORE_end,sup_TOTAL_DAMAGE_DEALT_end,sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end,sup_TOTAL_DAMAGE_TAKEN_end,inhibKills_end,towerKills_end,baronKills_end,dragonKills_end,total_pings
0,blue,2023-04-08,61881,63897,2016,13.6.500.291,MAD,Chasy,Elyoya,Nisqy,...,8,70.991379,21527.810547,4412.219727,12733.919922,0,3,0,2,603
0,red,2023-04-08,61881,63897,2016,13.6.500.291,VIT,Photon,Bo,Perkz,...,7,84.248413,18593.837891,5490.237793,11412.540039,1,7,1,2,780
0,blue,2023-04-08,68967,70780,1813,13.6.500.291,AST,Finn,113,LIDER,...,17,98.752251,14562.200195,4207.338867,10863.328125,0,3,0,0,517
0,red,2023-04-08,68967,70780,1813,13.6.500.291,FNC,Oscarinin,Razork,Humanoid,...,14,84.677071,16630.617188,3347.144775,9204.870117,2,10,2,4,343
0,blue,2023-04-09,61148,62895,1747,13.6.500.291,SK,Irrelevant,Markoon,Sertuss,...,10,78.173225,10635.515625,3330.705566,10101.532227,0,5,0,0,484


In [12]:
grouped_games = df.groupby("team")

In [17]:
def rolling_avg(group,cols,new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed ='left').mean() #assigns avg from previous 3 games onto the 4th game
    group[new_cols]=rolling_stats
    group = group.dropna(subset = new_cols) #drops the first 3 games without rolling averages
    return group

In [14]:
end_cols = [col for col in df.columns if "_end" in col]
cols = end_cols + ["total_pings","duration"]
new_cols = [f"{c}_rolling" for c in cols]

In [15]:
fnc = grouped_games.get_group("FNC")

In [18]:
rolling_avg(fnc,cols,new_cols)

,side,date,start_time,end_time,duration,patch,team,top,jng,mid,...,sup_VISION_SCORE_end_rolling,sup_TOTAL_DAMAGE_DEALT_end_rolling,sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling,sup_TOTAL_DAMAGE_TAKEN_end_rolling,inhibKills_end_rolling,towerKills_end_rolling,baronKills_end_rolling,dragonKills_end_rolling,total_pings_rolling,duration_rolling
0,blue,2023-03-18,68295,70349,2054,13.4.497.3450,FNC,Oscarinin,Razork,Humanoid,...,81.961739,16381.268880,7043.188477,7767.103516,0.000000,2.000000,0.000000,1.000000,240.333333,1767.333333
0,red,2023-03-19,69067,71581,2514,13.4.497.3450,FNC,Oscarinin,Razork,Humanoid,...,82.897575,17239.825521,6822.473307,8488.109701,0.000000,1.666667,0.000000,0.666667,257.000000,1844.000000
0,blue,2023-03-20,76337,78264,1927,13.4.497.3450,FNC,Oscarinin,Razork,Humanoid,...,84.427261,18688.234375,6442.276042,11190.843099,0.666667,4.666667,1.000000,1.666667,288.666667,2012.000000
0,blue,2023-03-25,72961,74874,1913,13.4.497.3450,FNC,Oscarinin,Razork,Humanoid,...,92.592440,20132.637370,5926.717204,11094.600911,1.666667,8.000000,1.666667,3.000000,352.333333,2165.000000
0,red,2023-03-26,71629,73601,1972,13.4.497.3450,FNC,Oscarinin,Razork,Humanoid,...,98.511393,18583.798828,4989.416097,11881.155599,2.666667,10.333333,2.000000,4.000000,411.666667,2118.000000
0,red,2023-03-27,61326,63560,2234,13.4.497.3450,FNC,Oscarinin,Razork,Humanoid,...,101.008265,16459.249349,4786.241618,10999.006836,3.333333,10.333333,1.666667,3.000000,393.666667,1937.333333
0,red,2023-04-08,68967,70780,1813,13.6.500.291,FNC,Oscarinin,Razork,Humanoid,...,102.226257,15771.320638,4941.147624,12551.476888,2.333333,8.000000,1.000000,2.000000,389.666667,2039.666667
0,red,2023-04-08,65649,67691,2042,13.6.500.291,FNC,Oscarinin,Razork,Humanoid,...,99.187515,15581.200846,4523.485270,12371.737956,2.000000,8.000000,1.333333,2.000000,344.666667,2006.333333
0,red,2023-04-08,71945,74682,2737,13.6.500.291,FNC,Oscarinin,Razork,Humanoid,...,89.323588,15702.537760,4200.809977,12534.466471,0.666667,4.666667,0.666667,2.333333,338.333333,2029.666667
0,red,2023-04-15,64335,66516,2181,13.6.500.291,FNC,Oscarinin,Razork,Humanoid,...,85.460955,16353.783529,4213.289958,12598.131836,0.666667,4.000000,0.666667,2.666667,342.666667,2197.333333


In [20]:
games_rolling = grouped_games.apply(lambda x:rolling_avg(x,cols,new_cols))
games_rolling

side        date  start_time  end_time  duration          patch team  \
team                                                                           
AST  0  blue  2023-03-18       61709     63507      1798  13.4.497.3450  AST   
     0   red  2023-03-19       61610     63691      2081  13.4.497.3450  AST   
     0  blue  2023-03-20       69302     71130      1828  13.4.497.3450  AST   
     0   red  2023-03-25       61778     63505      1727  13.4.497.3450  AST   
     0   red  2023-03-26       57510     58833      1323  13.4.497.3450  AST   
...      ...         ...         ...       ...       ...            ...  ...   
XL   0  blue  2023-03-19       65984     68100      2116  13.4.497.3450   XL   
     0  blue  2023-03-20       61692     63544      1852  13.4.497.3450   XL   
     0   red  2023-03-25       64640     67449      2809  13.4.497.3450   XL   
     0  blue  2023-03-26       57510     58833      1323  13.4.497.3450   XL   
     0   red  2023-03-27       57524     59887      2363  13.4.497.3450   XL   

            top    jng        mid  ... sup_VISION_SCORE_end_rolling  \
team                               ...                                
AST  0     Finn    113      LIDER  ...                    99.860382   
     0     Finn    113      LIDER  ...                    98.655492   
     0     Finn    113      LIDER  ...                   103.127960   
     0     Finn    113      LIDER  ...                    97.631955   
     0     Finn    113      LIDER  ...                    96.556076   
...         ...    ...        ...  ...                          ...   
XL   0  Odoamne  Xerxe     Vetheo  ...                    73.393163   
     0  Odoamne  Xerxe     Vetheo  ...                    86.863619   
     0  Odoamne  Xerxe  Abbedagge  ...                    93.714930   
     0  Odoamne  Xerxe  Abbedagge  ...                   105.404188   
     0  Odoamne  Xerxe  Abbedagge  ...                    87.490117   

       sup_TOTAL_DAMAGE_DEALT_end_rolling  \
team                                        
AST  0                       16794.774740   
     0                       18321.046224   
     0                       17414.978841   
     0                       18077.039388   
     0                       16736.099284   
...                                   ...   
XL   0                       24583.550130   
     0                       19667.931966   
     0                       17207.252930   
     0                       18576.560221   
     0                       20755.739583   

       sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling  \
team                                                     
AST  0                                     4427.193197   
     0                                     4855.921712   
     0                                     4188.577962   
     0                                     5731.230306   
     0                                     5050.963867   
...                                                ...   
XL   0                                     5918.723470   
     0                                     4607.079671   
     0                                     4706.559489   
     0                                     5500.595785   
     0                                     6004.481445   

       sup_TOTAL_DAMAGE_TAKEN_end_rolling inhibKills_end_rolling  \
team                                                               
AST  0                       12957.264160               0.666667   
     0                       15245.453776               0.333333   
     0                       12701.902018               0.000000   
     0                       12873.637370               0.666667   
     0                       11240.816081               1.000000   
...                                   ...                    ...   
XL   0                       16882.872070               0.000000   
     0                       15702.908854               0.333333   
     0                

In [21]:
games_rolling = games_rolling.droplevel('team')
games_rolling

,side,date,start_time,end_time,duration,patch,team,top,jng,mid,...,sup_VISION_SCORE_end_rolling,sup_TOTAL_DAMAGE_DEALT_end_rolling,sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling,sup_TOTAL_DAMAGE_TAKEN_end_rolling,inhibKills_end_rolling,towerKills_end_rolling,baronKills_end_rolling,dragonKills_end_rolling,total_pings_rolling,duration_rolling
0,blue,2023-03-18,61709,63507,1798,13.4.497.3450,AST,Finn,113,LIDER,...,99.860382,16794.774740,4427.193197,12957.264160,0.666667,5.666667,0.666667,1.666667,630.333333,1728.000000
0,red,2023-03-19,61610,63691,2081,13.4.497.3450,AST,Finn,113,LIDER,...,98.655492,18321.046224,4855.921712,15245.453776,0.333333,4.000000,0.333333,0.666667,568.000000,1754.666667
0,blue,2023-03-20,69302,71130,1828,13.4.497.3450,AST,Finn,113,LIDER,...,103.127960,17414.978841,4188.577962,12701.902018,0.000000,2.666667,0.000000,1.666667,596.333333,1781.333333
0,red,2023-03-25,61778,63505,1727,13.4.497.3450,AST,Finn,113,LIDER,...,97.631955,18077.039388,5731.230306,12873.637370,0.666667,5.333333,0.333333,2.333333,688.666667,1902.333333
0,red,2023-03-26,57510,58833,1323,13.4.497.3450,AST,Finn,113,LIDER,...,96.556076,16736.099284,5050.963867,11240.816081,1.000000,8.000000,0.666667,3.333333,745.666667,1878.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,blue,2023-03-19,65984,68100,2116,13.4.497.3450,XL,Odoamne,Xerxe,Vetheo,...,73.393163,24583.550130,5918.723470,16882.872070,0.000000,2.666667,0.333333,1.000000,352.333333,1890.666667
0,blue,2023-03-20,61692,63544,1852,13.4.497.3450,XL,Odoamne,Xerxe,Vetheo,...,86.863619,19667.931966,4607.079671,15702.908854,0.333333,4.000000,0.333333,1.666667,421.000000,1973.666667
0,red,2023-03-25,64640,67449,2809,13.4.497.3450,XL,Odoamne,Xerxe,Abbedagge,...,93.714930,17207.252930,4706.559489,13629.248861,0.333333,4.333333,0.333333,2.000000,448.333333,2077.666667
0,blue,2023-03-26,57510,58833,1323,13.4.497.3450,XL,Odoamne,Xerxe,Abbedagge,...,105.404188,18576.560221,5500.595785,11221.828939,1.000000,6.000000,0.666667,3.000000,520.333333,2259.000000


In [22]:
games = games_rolling

In [24]:
games["team_code"] = games["team"].astype("category").cat.codes
players = ["top","jng","mid","adc","sup","enemy_top","enemy_jng","enemy_mid","enemy_adc","enemy_sup"]
for col in players:
    games[f"{col}_code"] =  games[col].astype("category").cat.codes

In [27]:
games = games.sort_values("date")
games

,side,date,start_time,end_time,duration,patch,team,top,jng,mid,...,top_code,jng_code,mid_code,adc_code,sup_code,enemy_top_code,enemy_jng_code,enemy_mid_code,enemy_adc_code,enemy_sup_code
0,blue,2023-03-18,61709,63507,1798,13.4.497.3450,AST,Finn,113,LIDER,...,4,0,3,6,3,2,2,5,0,2
0,red,2023-03-18,61709,63507,1798,13.4.497.3450,MAD,Chasy,Elyoya,Nisqy,...,2,2,5,0,2,4,0,3,6,3
0,red,2023-03-18,75931,77814,1883,13.4.497.3450,G2,BrokenBlade,Yike,Caps,...,1,9,1,4,8,9,4,4,1,9
0,red,2023-03-18,72355,74227,1872,13.4.497.3450,SK,Irrelevant,Markoon,Sertuss,...,5,5,8,3,1,8,1,6,9,4
0,blue,2023-03-18,68295,70349,2054,13.4.497.3450,FNC,Oscarinin,Razork,Humanoid,...,7,6,2,8,0,0,7,10,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,red,2023-04-23,70823,72520,1697,13.6.500.291,MAD,Chasy,Elyoya,Nisqy,...,2,2,5,0,2,0,7,10,2,6
0,blue,2023-04-23,58511,60536,2025,13.6.500.291,BDS,Adam,Sheo,nuc,...,0,7,10,2,6,2,2,5,0,2
0,blue,2023-04-23,68130,69577,1447,13.6.500.291,BDS,Adam,Sheo,nuc,...,0,7,10,2,6,2,2,5,0,2
0,red,2023-04-23,65139,66915,1776,13.6.500.291,BDS,Adam,Sheo,nuc,...,0,7,10,2,6,2,2,5,0,2


In [28]:
games.to_csv("lec_spring_2023.csv",index=False)